In [1]:
STOCK_NAME = '카카오'

In [2]:
import numpy as np
import os
import pickle
import random
import datetime
from tqdm import tqdm
from multiprocessing import Pool
from MeCab import Tagger
import tensorflow as tf
import tensorflow.keras.backend as ktf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, Concatenate, Input, Embedding, Reshape
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

## GPU Setting

In [5]:
# gpu 설정
def get_session():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5,allow_growth=True,visible_device_list='1')
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
ktf.set_session(get_session())

## Hyper Parameters

In [6]:
# 하이퍼파라미터
seq_size = 70  # model input shape[0], 입력 데이터의 형태소 최대 갯수
embed_size = 128  # model input shape[1], 각 형태소의 임베딩 벡터 크기
batch_size = 32 # 각 미니배치의 크리
vocab_size = 455001  # word2index_dict의 단어 수
validation_split = 0.1  # 학습 시 train set에서 validation set으로 사용할 데이터 비율
learning_rate = 0.001
dropout_rate = 0.5
kernel_sizes = [3, 4, 5]  # kernel_size list for each CNN layers
n_class = 1  # 분류할 클래스 수 (binary_crossentropy를 쓰므로 1개 클래스가 0 또는 1의 값을 가지는 것으로 2 클래스 분류)
n_epoch = 20
n_patience = 6  # early stop 콜백의 파라미터

# random seed
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

## 경로 설정

In [7]:
# 파일 경로
root_path = '/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification'
data_path = f'{root_path}/data/sentences/increased_labeled/labeled_{STOCK_NAME}.txt'
vocab_path = f'{root_path}/data/vocabulary/word2index_dict_190117.pkl'
mecab_path = '-d /data/lib/mecab/mecab-ko-dic-default'

# 저장경로용 시간, 파일명 문자열
now_dt = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
model_name = data_path.split('/')[-1].replace('labeled_', '').replace('.txt', '')

# 텐서보드 디렉토리 생성 및 로그 경로
tensorboard_dir = 'tensorboard'
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)
tblog_path = f'{root_path}/{tensorboard_dir}/{model_name}'

# 체크포인트 디렉토리 생성
ckp_dir = f'{root_path}/checkpoint/{model_name}/'
if not os.path.exists(ckp_dir):
    os.makedirs(ckp_dir)
ckp_path = os.path.join(ckp_dir, now_dt + '_weights.{epoch:03d}-{val_acc:.4f}.hdf5')

## 단어사전 로드
word2index_dict.pkl을 로드한다. 거의 대부분의 단어들을 유니크한 숫자로 인코딩하기 위한 딕셔너리이다. (str -> int)

In [8]:
with open(vocab_path, 'rb') as fp:
    word2index_dict = pickle.load(fp)
    
if vocab_size != len(word2index_dict):
    vocab_size = len(word2index_dict)
print(len(word2index_dict))

455001


## 데이터 로드


In [7]:
def load_data(path):
    with open(data_path, 'r') as fp:
        data = [l.strip() for l in fp.readlines() if len(l) > 10 and len(l.split())]
    data_X = [d.rsplit('#', 1)[0] for d in data]
    data_y = [int(d.rsplit('#', 1)[-1]) for d in data]
    return data_X, data_y

In [8]:
data_X, data_y = load_data(data_path)
print('data_X size : ', len(data_X))
print('data_y size : ', len(data_y))
print('data_X[10] : ', data_X[10])
print('data_y[10] : ', data_y[10])

data_X size :  8040
data_y size :  8040
data_X[10] :  이들은 마약 성분으로 유명한 코카 대신 초콜릿 원료인 카카오를 키우고 있다
data_y[10] :  0


### !!! 클래스별 데이터 수 비교하기



In [9]:
# 레이블별 데이터 비율 비교
class_0_data = [x for x, y in zip(data_X, data_y) if y == 0]
class_1_data = [x for x, y in zip(data_X, data_y) if y == 1]

print(f'class_0_data size : {len(class_0_data)}')
print(f'class_1_data size : {len(class_1_data)}')

class_0_data size : 3270
class_1_data size : 4770


In [10]:
# sample_x_c0 = class_0_data
# sample_x_c1 = random.sample(class_1_data, 40000)
# data_X = sample_x_c0 + sample_x_c1

## Tokenizing

In [9]:
def tokenize(sentence):
    tagger = Tagger(mecab_path)
    raw_tokens = tagger.parse(sentence).splitlines()
    parsed_tokens = []
    for raw_token in raw_tokens:
        word_tag_ruple = raw_token.split('\t')
        if len(word_tag_ruple) != 2:
            continue
        word_stem = word_tag_ruple[0]
        word_tag = word_tag_ruple[1].split(',')[0]
        if not word_stem or not word_tag:
            continue
        if word_tag in {'NNP', 'NNG', 'SL', 'VV', 'VA', 'VX'}:
            parsed_tokens.append(word_stem)
    return parsed_tokens

In [12]:
tokenized_sents = []
with Pool(processes=8) as pool:
    for tokens in tqdm(pool.imap(tokenize, data_X), total=len(data_X), desc='tokenizing'):
        if tokens:
            tokenized_sents.append(tokens)
            
print('tokenized_sents size : ', len(tokenized_sents))

tokenizing: 100%|██████████| 8040/8040 [00:01<00:00, 5649.92it/s]


tokenized_sents size :  8040


## Encoding

In [10]:
def encode(tokens):
    padded_tokens = list(map(lambda x : tokens[x] if x < len(tokens) else '#', range(seq_size)))
    embed_vect = list(map(lambda w : word2index_dict[w] if w in word2index_dict else 0, padded_tokens))
    return embed_vect

In [14]:
encode_X = []
with Pool(processes=8) as pool:
    for encoded_tokens in tqdm(pool.imap(encode, tokenized_sents), total=len(data_X), desc='encoding'):
        encode_X.append(encoded_tokens)
        
print('encode_X size : ', len(encode_X))

encoding: 100%|██████████| 8040/8040 [00:00<00:00, 14093.46it/s]


encode_X size :  8040


## Split Train - Test Dataset

In [15]:
def get_train_test_data(data_x, data_y):
    
    x_c0, x_c1 = [], []
    for x, y in zip(data_x, data_y):
        if y == 0:
            x_c0.append(x)
        if y == 1:
            x_c1.append(x)
    
    increased_c1 = []
    
    if len(x_c0) >= len(x_c1)*2:
        div_ratio = int(len(x_c0) / len(x_c1))
        for i in range(div_ratio):
            if  (i + 1) * len(x_c1) < len(x_c0):
                increased_c1 += x_c1
    else:
        increased_c1 = x_c1
            
    shuff_c0 = random.sample(x_c0, len(x_c0))
    shuff_c1 = random.sample(increased_c1, len(increased_c1))
    
    c0_tsize = int(len(shuff_c0) * validation_split)
    c1_tsize = int(len(shuff_c1) * validation_split)
    
    test_X = shuff_c0[:c0_tsize] + shuff_c1[:c1_tsize]
    train_X = shuff_c0[c0_tsize:] + shuff_c1[c1_tsize:]
    test_y = [0 for _ in range(c0_tsize)] + [1 for _ in range(c1_tsize)]
    train_y = [0 for _ in range(len(shuff_c0) - c0_tsize)] + [1 for _ in range(len(shuff_c1) - c1_tsize)]    
        
    return np.array(train_X), np.array(train_y), np.array(test_X), np.array(test_y)


In [16]:
train_X, train_y, test_X, test_y = get_train_test_data(encode_X, data_y)

print(f'train_X size : {len(train_X)}')
print(f'train_y size : {len(train_y)}')
print(f'test_X size : {len(test_X)}')
print(f'test_y size : {len(test_y)}')

train_X size : 7236
train_y size : 7236
test_X size : 804
test_y size : 804


## Model

In [17]:
def get_char_cnn_model(kernel_sizes, seq_size, vocab_size, embed_size, dropout_rate, n_class, learning_rate):
    # input layer
    inputs = Input(shape=(seq_size,), dtype='float32', name='input')
    
    embedded = Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_size, name='embedded')(inputs)
    reshaped = Reshape((seq_size, embed_size, 1), name='reshape')(embedded)

    # multiple CNN layers
    convolution_layers = []
    for idx, kernel_size in enumerate(kernel_sizes):
        conv = Conv2D(filters=256, kernel_size=(kernel_size, embed_size), padding='valid', activation='relu', name=f'conv_{idx}')(reshaped)
        pool = MaxPooling2D(pool_size=(seq_size - kernel_size + 1, 1), strides=(1, 1), padding='valid', name=f'pool_{idx}')(conv)
        dropout = Dropout(rate=dropout_rate, name=f'dropout_{idx}')(pool)
        convolution_layers.append(dropout)
        
    # concatenate all CNN output tensors
    concat_tensor = Concatenate(axis=1, name='concat')(convolution_layers)    
    
    # middle fully-connected layer
    flatten = Flatten(name='flatten')(concat_tensor)
    hidden = Dense(384, activation='relu', name='hidden')(flatten)
    dropout = Dropout(rate=dropout_rate, name='dropout')(hidden)
    
    # output layer
    outputs = Dense(n_class, activation='sigmoid', name='output')(dropout)

    # connect layers to model
    model = Model(inputs=inputs, outputs=outputs)
    
    # compile model
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
model = get_char_cnn_model(kernel_sizes, seq_size, vocab_size, embed_size, dropout_rate, n_class, learning_rate)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 70)           0                                            
__________________________________________________________________________________________________
embedded (Embedding)            (None, 70, 128)      58240128    input[0][0]                      
__________________________________________________________________________________________________
reshape (Reshape)               (None, 70, 128, 1)   0           embedded[0][0]                   
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 68, 1, 256)   98560       reshape[0][0]                    
__________________________________________________________________________________________________
conv_1 (Co

## Callbacks

In [19]:
early_stopping = EarlyStopping(patience=n_patience)
# tensorboard = TensorBoard(log_dir=tblog_path, batch_size=batch_size)
checkpoint = ModelCheckpoint(ckp_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_weights_only=False)

In [20]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir=tblog_path, **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()
        
tensorboard = TrainValTensorBoard(batch_size=batch_size)

## Training

In [21]:
history = model.fit(
    train_X, 
    train_y, 
    batch_size=batch_size, 
    epochs=n_epoch,
    validation_data=(test_X, test_y),
    callbacks=[early_stopping, tensorboard, checkpoint]
)

/data/venv/lib64/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7236 samples, validate on 804 samples
Epoch 1/20
7236/7236 [==============================] - 7s 999us/step - loss: 0.5373 - acc: 0.7062 - val_loss: 0.1464 - val_acc: 0.9565

Epoch 00001: val_loss improved from inf to 0.14640, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/카카오/20190308142943_weights.001-0.9565.hdf5
Epoch 2/20
7236/7236 [==============================] - 6s 775us/step - loss: 0.1202 - acc: 0.9588 - val_loss: 0.0691 - val_acc: 0.9751

Epoch 00002: val_loss improved from 0.14640 to 0.06909, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/카카오/20190308142943_weights.002-0.9751.hdf5
Epoch 3/20
7236/7236 [==============================] - 6s 775us/step - loss: 0.0667 - acc: 0.9811 - val_loss: 0.0581 - val_acc: 0.9826

Epoch 00003: val_loss improved from 0.06909 to 0.05814, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/카카오/20190308142943_

## Predict

In [11]:
loaded_model = tf.keras.models.load_model('/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/카카오/20190308142943_weights.003-0.9826.hdf5')

OSError: Unable to open file (unable to open file: name = '/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/카카오/20190308142943_weights.003-0.9826.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [134]:
def predict(model, sentence):
    tokens = tokenize(sentence)
    encodes = encode(tokens)
    tf_input = np.array(encodes).reshape((1, seq_size))
    pred = model.predict(tf_input)[0][0]
    result = 1 if pred >= 0.85 else 0
    return result, pred

In [135]:
text = '카카오IX는 카카오프렌즈를 활용한 스낵 브랜드 선데이치즈볼의 첫 번째 팝업스토어(임시매장)가 현대백화점 판교점에 입점한다고 18일 밝혔다.'
pred = predict(loaded_model, text)
pred

(1, 0.99480647)

In [136]:
text = '[5시뉴스] ◀ 앵커 ▶ 카카오 카풀 시범 서비스 중단 사흘 만에, 택시업계가 사회적 대타협 기구에... 카카오가 카풀 시범 서비스 중단을 발표한 지 사흘만입니다.'
pred = predict(loaded_model, text)
pred

(1, 0.9615411)

In [137]:
text = '(서울=연합인포맥스) 김경림 기자 = 카카오M이 디지털 콘텐츠 분야의 경력직원과 인턴을 모집한다고 18일 발표했다.'
pred = predict(loaded_model, text)
pred

(1, 0.99309796)

In [138]:
text = '[폴리뉴스 조민정 기자] 카카오게임즈가 지난 17일 서비스 중인 모바일 액션 RPG ‘외모지상주의’에 신규 캐릭터 ‘최수정’을 업데이트했다.'
pred = predict(loaded_model, text)
pred

(1, 0.9996935)

In [139]:
text = '카카오닙스는 달콤한 초콜릿과 달리 향은 비슷하나 쓴맛이 나며 씹으면 씹을 수록 고소한 맛이 나는 것이 특징이다.'
pred = predict(loaded_model, text)
pred

(0, 5.399636e-05)

In [140]:
text = '카카오닙스는 초콜렛을 만드는 원재료인 카카오열매의 껍질을 제거하여 코코아를 꺼낸 후 말려 부수어 먹기 좋게 만들어낸 식품을 말한다'
pred = predict(loaded_model, text)
pred

(0, 0.020765064)

In [141]:
text = '뉴 C4 칵투스를 국내 공식 출시 전 특별 전시하며, 초콜릿 상자처럼 랩핑된 뉴 C4 칵투스가 초콜릿 드레스 패션쇼 카카오쇼 무대 옆에 위치할 예정이다'
pred = predict(loaded_model, text)
pred

(0, 5.31054e-05)

In [142]:
text = '이 제품은 쁘띠 몽쉘 크림과 쁘띠 몽쉘 카카오 등 2종으로 구성된다. 무엇보다도 생크림 함량을 국내 최고 수준인 3.4%로 높여 더 진한 크림 맛을 즐길 수 있다는 것이 롯데제과의 설명이다.'
pred = predict(loaded_model, text)
pred

(0, 0.016149683)

In [143]:
text = '벨기에산 카카오 크림 등으로 만든 미니 케이크 등을 판매한다.'
pred = predict(loaded_model, text)
pred

(0, 0.0063802567)

In [144]:
text = ' 현재 롯데제과는 Δ크림케이크 Δ카카오케이크 Δ요거트&블루베리...'
pred = predict(loaded_model, text)
pred

(0, 0.022012075)

In [145]:
text = "이 자전거를 체험해보기위해 휴대전화로 전용 애플리케이션(앱·APP) '카카오T'를 실행했다"
pred = predict(loaded_model, text)
pred

(1, 0.98922694)

In [146]:
text = "모바일 간편결제 서비스인 '카카오페이'를 통해 보증금 1만원을 결제하니 휴대전화 화면에 지도와 함께 카카오T 바이크 위치가 표시됐다"
pred = predict(loaded_model, text)
pred

(1, 0.99998426)

In [147]:
text = '법무부가 ‘카카오톡 플러스친구’를 통해 국적관련 정보를 재외국민들에게 공지하고 있다'
pred = predict(loaded_model, text)
pred

(0, 0.004509998)

In [148]:
text = '법무부는 “미국, 캐나다 등 해외에서 거주하는 동포들이 국적관련 신고 시기를 놓쳐 어려움을 호소하는 사례가 종종 발생하고 있다”면서 “법령에 대한 이해부족으로 국적관련 신고기간을 놓치는 사례가 없도록 하기 위해 현지 교민단체 등을 통해 직접 홍보하는 한편, 지난해 11월 ‘카카오톡 플러스친구’를 개설해 운영하고 있다”고 밝혔다'
pred = predict(loaded_model, text)
pred

(0, 0.0027518598)

In [149]:
text = '현재 법무부 카카오톡 플러스친구에 친구 맺기를 한 사람은 약 1만2천여명'
pred = predict(loaded_model, text)
pred

(0, 0.0015177563)

In [150]:
text = '카카오톡을 통해 법무부는 국적선택, 국적이탈, 국적상실, 국적보유 등 재외국민이 꼭 알아야 할 신고에 대한 유의사항을 안내하고 있다'
pred = predict(loaded_model, text)
pred

(0, 0.21820006)

In [151]:
text = '이디야커피(대표 문창기)가 카카오IX(대표 권승조)의 캐릭터 브랜드 카카오프렌즈와의 콜라보레이션 제품을 오는 12일 출시한다'
pred = predict(loaded_model, text)
pred

(1, 0.9361771)

In [152]:
text = '이디야커피는 카카오프렌즈와 함께 올 한 해 동안 시즌 제품 및 MD상품을 지속적으로 출시할 계획이라고 밝혔다'
pred = predict(loaded_model, text)
pred

(1, 0.9851281)

In [153]:
text = '커피 프랜차이즈 이디야커피는 카카오프렌즈와 콜라보레이션을 통해 음료 제품을 출시하는 것으로 고객들은 올해 전국 2,500여개의 이디야커피 매장에서 국내 최고의 인기 캐릭터 콜라보레이션 제품을 만나 볼 수 있다'
pred = predict(loaded_model, text)
pred

(0, 0.2161111)

In [154]:
text = '카카오 프렌즈 피규어가 포함된 ‘츄파춥스+카카오 피규어’ 상품은 1980원에, ‘롯데 라인프렌즈 캔디/젤리’는 3980원/9980원에, ‘디즈니 겨울왕국 틴’은 1만2980원에 판매한다'
pred = predict(loaded_model, text)
pred

(1, 0.9659929)

In [155]:
text = '이들이 귀중히 여겼던 카카오 콩은 유럽에 전해진 뒤 19세기 초, 네덜란드인 반 호텐이 설탕을 섞어 지금과 같은 초콜릿을 만들어내면서 급속히 퍼졌다'
pred = predict(loaded_model, text)
pred

(0, 0.0001641676)

In [156]:
text = '코트디부아르 영부인인 도미니크 와타라 여사가 2016년 카카오 농장에서의 어린이 노동에 대해 연설하고 있다'
pred = predict(loaded_model, text)
pred

(0, 0.38210738)

In [157]:
text = '롯데제과 몽쉘 생크림케이크 오리지널(왼쪽), 몽쉘 생크림케이크 카카오. (사진=롯데제과)'
pred = predict(loaded_model, text)
pred

(0, 0.008417338)

In [158]:
text = '쁘띠 몽쉘은 맛에 따라 ‘쁘띠몽쉘 크림’과 ‘쁘띠몽쉘 카카오’ 2종으로 출시됐으며 기존 몽쉘에 바닐라빈과 헤이즐넛 커피잼 등 새로운 소재를 첨가해 맛을 업그레이드 했다'
pred = predict(loaded_model, text)
pred

(0, 0.017779905)

In [159]:
text = '쁘띠몽쉘 크림은 크림 속에 바닐라빈을 넣어 부드럽고 달콤한 맛을 강조한 제품이며 쁘띠몽쉘 카카오는 달콤한 초코크림 속에 헤이즐넛 향의 커피잼을 넣은 것이 특징이다'
pred = predict(loaded_model, text)
pred

(0, 0.0033764485)

In [160]:
text = "이 제품은 '쁘띠 몽쉘 크림'과 '쁘띠 몽쉘 카카오' 등 2종으로 구성된다"
pred = predict(loaded_model, text)
pred

(0, 0.023029672)

In [161]:
text = "또 '쁘띠 몽쉘 카카오'는 초코크림 속에 헤이즐넛 향의 커피 잼을 넣어 커피 맛을 내는 것이 특징이다"
pred = predict(loaded_model, text)
pred

(0, 0.0034910764)

In [162]:
text = "카카오닙스성분에는 '폴리페놀'이 들어있다. "
pred = predict(loaded_model, text)
pred

(0, 0.003032277)

In [163]:
text = '폴리페놀 성분은 카카오닙스 외에도 녹차나 일반 초콜렛에서 만나볼 수 있는 성분이다. '
pred = predict(loaded_model, text)
pred

(0, 0.0009998393)

In [164]:
text = '카카오닙스에는 식이섬유가 많이 들어있는데, 이는 운동량이 부족하며 싱싱한 과일, 야채의 섭취를 잘 하지 못하여 소화에 불편을 겪는 현대인들에게 적합하다. '
pred = predict(loaded_model, text)
pred

(0, 0.17026633)

In [165]:
text = '카카오 열매들이 자라고 있는 모습. 위키미디아 커먼스 제공 '
pred = predict(loaded_model, text)
pred

(0, 0.6173388)

In [166]:
text = '초콜릿의 주원료는 카카오나무 열매의 씨앗인 카카오 콩'
pred = predict(loaded_model, text)
pred

(0, 0.0009354883)

In [167]:
text = '이 콩은 15세기 말 이탈리아의 탐험가 크리스토퍼 콜럼버스가 중앙아메리카에서 카카오 열매를 가지고 돌아오면서 유럽에 처음 전해졌다'
pred = predict(loaded_model, text)
pred

(0, 0.09065647)

In [168]:
text = '벨기에 국립은행 박물관에 따르면 카카오나무를 처음 기른 것은 고대 마야인들'
pred = predict(loaded_model, text)
pred

(0, 0.46959686)

In [169]:
text = '기원전 1500년경부터 약 3000년 동안 멕시코와 과테말라를 중심으로 중앙아메리카 대륙에서 번성했던 마야 문명의 사람들은 카카오 콩을 음식이나 옷으로 교환하며 화폐로 사용하거나 카카오 음료를 약으로 복용했다'
pred = predict(loaded_model, text)
pred

(0, 0.0297495)